In [2]:
import sys
sys.path.append('../')

In [3]:
import OpenCity
import PreCompOsmNet

In [4]:
import osmnx

## Download geometry and OD information for a US State

In [5]:
fips=25
year=2017
geom_type='block'
centre= {'lat': 42.352927, 'lon': -71.059435}
centre_x_y=[centre['lon'], centre['lat']]

model_area_radius=5000
sim_area_radius=1000

state=OpenCity.US_State(state_fips=fips, year=year, 
#                         geom_type=geom_type
                       )
state.get_geometry()

AttributeError: module 'OpenCity' has no attribute 'US_State'

## Specify which zones are in (i) the model area (full study area) and (ii) the simulation area (bounds of the table)

In [ ]:
state.subset_geom_by_distance(centre_x_y, model_area_radius, 'model_area')
state.subset_geom_by_distance(centre_x_y, sim_area_radius, 'sim_area')

In [ ]:
all_zones=state.return_geometry()
sim_zones=state.return_geometry('sim_area')
model_zones=state.return_geometry('model_area')

## Simulate a population based on the OD matrix
### Sim Pop will only include individuals who satisfy both of these conditions:
- live AND work in the model area
- live OR work in the sim area. 

In [ ]:
state.get_lodes_data()
simpop_df=state.lodes_to_pop_table(model_subset_name='model_area',
                                  sim_subset_name='sim_area')

## Create a osmnx networks for each mode and pre-comoute routes between all node pairs for faster route finding at simulation time

In [ ]:
osmnx.utils.config(bidirectional_network_types=['walk', 'drive', 'cycle'])
print('Getting walk network')
G_walk = osmnx.graph.graph_from_point([centre['lat'], centre['lon']], 
                                      dist=sim_area_radius, network_type='walk')
for edge in list(G_walk.edges):
    G_walk.edges[edge]['speed_kph']=4.8
G_walk=PreCompOsmNet.simplify_network(G_walk)
G_walk=osmnx.add_edge_travel_times(G_walk)
fw_pred_walk=PreCompOsmNet.pre_compute_paths(G_walk)
pre_comp_walk=PreCompOsmNet.PreCompOSMNet(G_walk, fw_pred_walk)

print('Getting drive network')
G_drive = osmnx.graph.graph_from_point([centre['lat'], centre['lon']], 
                                       dist=sim_area_radius, network_type='drive')
G_drive = osmnx.add_edge_speeds(G_drive)
G_drive=PreCompOsmNet.simplify_network(G_drive)
# compute speeds
G_drive=osmnx.add_edge_travel_times(G_drive)
fw_pred_drive=PreCompOsmNet.pre_compute_paths(G_drive)
pre_comp_drive=PreCompOsmNet.PreCompOSMNet(G_drive, fw_pred_drive)

In [ ]:
networks={'drive': pre_comp_drive, 'walk': pre_comp_walk}

drive_dict={
    'target_network_id': 'drive',
    'travel_time_metric': 'travel_time'}
walk_dict={
    'target_network_id': 'walk',
    'travel_time_metric': 'travel_time'}

modes={'drive': OpenCity.Mode(drive_dict), 
       'walk': OpenCity.Mode(walk_dict)}

mob_sys=OpenCity.MobilitySystem(modes=modes, networks= networks)

# Create the Simulation

In [ ]:
sim=OpenCity.Simulation(simpop_df, mob_sys, model_zones)
simpop_df=sim.get_simpop_subset(simpop_df, sample_N=1000)
simpop_df=sim.create_simple_HWH_schedules(simpop_df)
all_trips_df=sim.create_trip_table(simpop_df)
all_trips_df=sim.mode_chooser(all_trips_df)
route_table=sim.get_routes_table(all_trips_df)
route_gdf=sim.route_table_to_geo(route_table)

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(1, figsize=(12,12))
route_gdf.plot(column='earnings', legend=True, alpha=0.1, ax=ax)